In [1]:
import nxsdk.api.n2a as nx
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle as p

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from nxsdk.graph.monitor.probes import PerformanceProbeCondition
from nxsdk.api.enums.api_enums import ProbeParameter

In [3]:
import blackjackAgent
from analysis_functions import *

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)


The most difficult problem the agent framework is currently demonstrated with is the card game 'blackjack.' In this game, the goal is to draw cards totaling 21 points without exceeding this limit. See [this article](https://en.wikipedia.org/wiki/Blackjack#Rules) for an overview of the rules. 

In [4]:
#in blackjack, all face cards (Jack, Queen, King) are scored the same so we are effectively left with 10 cards
n_cards = 10

#aces can be counted as 1 or 10 points, doubling the number of states in the game
n_states = n_cards**2 * 2
#a player can either hit (draw a card) or stick (refuse another card and 
# use the current sum to play against the dealer to end the round)
n_actions = 2
n_estimates = n_states * n_actions

In [5]:
l_epoch = 128
n_replicates = 2
episodes = 10000
runs = 5

In [6]:
#we will give the agent an initial policy to stick on a player sum of 20 or 21.
init_vals = np.zeros((n_actions,2,10,10,n_replicates),dtype=np.int)
init_vals[0,:,:,8:,:] = -1.0
init_vals[1,:,:,8:,:] = 1.0
init_vals *= (2**6) * 127

In [11]:
agent = blackjackAgent.BlackjackAgent(n_actions, 
                                      n_states, 
                                      n_replicates=n_replicates, 
                                      l_epoch=l_epoch, 
                                      n_epochs=episodes,
                                      starting_values = init_vals,
                                      probe = True)

In [12]:
results = []

for i in range(1):
    results.append(agent.run())
agent.board.disconnect()

INFO:DRV:  SLURM is being run in background
INFO:HST:  srun: error: Unable to allocate resources: Requested node configuration is not available
INFO:DRV:      Host server up..............Done 3.66s


KeyboardInterrupt: 

We can inspect the state and action pairs which were taken by the agent during the learning process. Note that the action always comes after the state and categories in the table below are only arranged alphebetically. 

In [10]:
final_run = results[-1]
history = read_history(final_run[0])
history[-10:]

KeyError: 14

To inspect the policy which has been learned by the agent after play, we can look at the final on-chip estimates of value which are read back to the host after the run has been completed. 

In [ ]:
fe = agent.final_estimates;
policy = data_to_policy({"final_estimates" : [agent.final_estimates], "conditions" : {"dynrange" : 1}})

In [ ]:
optimal = open_julia_single("optimal.jld2","optimal")

Because there are only 2 actions in Blackjack (stick & hit), we can inspect policy by looking at the difference in value between the two. Below, we plot the learned difference in value between sticking & hitting. We expect this value to be very high when player sum is already close to 20 & 21, and this is observed. More advanced strategies for lower player sums also depend on the dealer card.

In [ ]:
#Loihi values after 5e4 episodes
plot_policy(policy[0])

In [ ]:
#CPU values after 2e6 episodes
plot_policy(optimal)

As a greedy policy is being used, we can use this value difference to directly calculate the policy. Anywhere the difference is greater than 0, the 'stick' action will be chosen. At this point (post 50,000 games) the agent is still approaching the optimal policy, but the gross aspects of the strategy (sticking on high values) are in place.

In [ ]:
#Loihi policy after 5e4 episodes
plot_policy(policy[0] > 0.00)

In [ ]:
#CPU policy after 2e6 episodes
plot_policy(optimal > 0.00)